In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Combine the fbref big-5 league data into a single dataframe. And standardise stats to per 90.

Cycle through the scraped fbref data and where there is more than one record per player sum the stats (if relevant)

In [2]:
files = [['fbref_playingtime-18.parquet', 'fbref_playingtime-19.parquet', 'fbref_playingtime-20.parquet'],
         ['fbref_defence-18.parquet', 'fbref_defence-19.parquet', 'fbref_defence-20.parquet'],
         ['fbref_gca-18.parquet', 'fbref_gca-19.parquet', 'fbref_gca-20.parquet'],
         ['fbref_misc-18.parquet', 'fbref_misc-19.parquet', 'fbref_misc-20.parquet'],
         ['fbref_passing-18.parquet', 'fbref_passing-19.parquet', 'fbref_passing-20.parquet'],
         ['fbref_passing_types-18.parquet', 'fbref_passing_types-19.parquet', 'fbref_passing_types-20.parquet'],
         ['fbref_possession-18.parquet', 'fbref_possession-19.parquet', 'fbref_possession-20.parquet'],
         ['fbref_shooting-18.parquet', 'fbref_shooting-19.parquet', 'fbref_shooting-20.parquet'],
         ['fbref_stats-18.parquet', 'fbref_stats-19.parquet', 'fbref_stats-20.parquet']]

# dropping these columns as either impossible to re-create as averages and need to recreate from the summed stats
drop_list = ['playing_time_min%', 'starts_mn/start', 'subs_mn/sub', 'team_success_(xg)_on-off',
             'team_success_on-off', 'team_success_ppm', 'playing_time_mn/mp',
             'standard_g/sh', 'standard_g/sot', 'expected_npxg/sh']

df_list = []
for file in files:
    dfs = [pd.read_parquet(os.path.join('data', 'fbref', f)) for f in file]
    
    # only keep 2018 and 2019 results if the player played in 2020-21
    dfs[0] = dfs[0][dfs[0].player_link.isin(dfs[2].player_link)]
    dfs[1] = dfs[1][dfs[1].player_link.isin(dfs[2].player_link)]
    df = pd.concat(dfs)
    
    # split players into one club/ multi club
    df_one_club = df.drop_duplicates('player_link', keep=False).copy()
    df_multi_club = df[df.duplicated('player_link', keep=False)]
    
    # split columns into groups (per 90 stats/ percentage stats, stats that sum)
    cols = (df.columns[8:-2])
    cols90 = [c for c in cols if '90' in c and c != '90s']
    colspct = [c for c in cols if '%' in c]
    drop_list = drop_list + cols90 + colspct
    cols_to_sum = [c for c in cols if c not in drop_list]
    
    # sum up the stats that sum. where a player has been at multiple clubs we are going to sum the stats over these clubs
    df_multi_club_sum = df_multi_club.groupby('player_link')[cols_to_sum].sum().reset_index()
    df_multi_club = (df_multi_club[['rk', 'player', 'nation', 'pos', 'squad', 'comp', 'age', 'born', 
                                    'player_link', 'match_link']].drop_duplicates('player_link', keep='last'))
    df_multi_club = df_multi_club.merge(df_multi_club_sum, on='player_link', validate='1:1')
    
    # drop cols that don't sum
    df_one_club.drop(drop_list, axis='columns', errors='ignore', inplace=True)
    
    # concatenate back with the players with only one club
    df = pd.concat([df_one_club, df_multi_club])
    df_list.append(df)

Merge the dataframes in the list into one dataframe and drop some repeated columns

In [3]:
df = df_list[0]
for i, df_join in enumerate(df_list[1:]):
    df = df.merge(df_join[df_join.columns[8:]], on='player_link', validate='1:1', suffixes=['', f'_{i}'])

# drop some columns
drop_cols = [c for c in df.columns if c[-1].isnumeric() and c[-2] == '_']
df.drop(drop_cols, axis='columns', inplace=True)
cols_repeated = ['att', 'outcomes_cmp',  'expected_xa', 'playing_time_starts', 
                 'performance_tklw', 'performance_ast', 'performance_int', 'performance_crs',
                 'performance_pk', 'performance_pkatt', 'performance_gls', 'performance_pk', 'performance_pkatt']
df.drop(cols_repeated, axis='columns', inplace=True)
df.drop(['rk', 'standard_dist', 'age'], axis='columns', inplace=True)

# reorder cols
str_cols = [c for c in df.columns if (df[c].dtype == object) | (c == '90s')]
other_cols = [c for c in df.columns if (df[c].dtype != object) & (c != '90s')]
col_order = str_cols + other_cols
df = df[col_order].copy()

Get average shot distance (can't just sum as it's already averaged)

In [4]:
df_shots = pd.concat([pd.read_parquet(os.path.join('data', 'fbref', f))[['player_link',
                                                                         'standard_dist', 'standard_sh']] for f in files[7]])
df_shots.fillna(0, inplace=True)
df_shots['distance'] = df_shots.standard_dist * df_shots.standard_sh
df_shots = df_shots.groupby(['player_link'])[['standard_sh', 'distance']].sum().reset_index()
df_shots['standard_dist'] = df_shots['distance'].divide(df_shots['standard_sh']).round(1).fillna(0)
df_shots = df_shots[['player_link', 'standard_dist']].copy()
df = df.merge(df_shots, how='left', on='player_link')

Add the percent/ ratio columns

In [5]:
df['vs_dribbles_tkl%'] = (df['vs_dribbles_tkl'].divide(df['vs_dribbles_att']) * 100.).round(1)
df['pressures_%'] = (df['pressures_succ'].divide(df['pressures_press']) * 100.).round(1)
df['aerial_duels_won%'] =  (df['aerial_duels_won'].divide(df['aerial_duels_won'] + df['aerial_duels_lost']) * 100.).round(1)
df['total_cmp%'] =  (df['total_cmp'].divide(df['total_att']) * 100.).round(1)
df['short_cmp%'] =  (df['short_cmp'].divide(df['short_att']) * 100.).round(1)
df['medium_cmp%'] =  (df['medium_cmp'].divide(df['medium_att']) * 100.).round(1)
df['long_cmp%'] =  (df['long_cmp'].divide(df['long_att']) * 100.).round(1)
df['playing_time_mn/mp'] =  (df['playing_time_min'].divide(df['playing_time_mp'])).round(1)
df['dribbles_succ%'] =  (df['dribbles_succ'].divide(df['dribbles_att']) * 100.).round(1)
df['receiving_rec%'] =  (df['receiving_rec'].divide(df['receiving_targ']) * 100.).round(1)
# goal ratio columns
non_penalty_sh = df['standard_sh'] - df['standard_pkatt']
non_penalty_goals = df['standard_gls'] - df['standard_pk']
df['standard_g/sh'] =  non_penalty_goals.divide(non_penalty_sh).round(2)
df['expected_npxg/sh'] =  df['expected_npxg'].divide(non_penalty_sh).round(2)
df['standard_g/sot'] = non_penalty_goals.divide(df['standard_sot']).round(2)
df['standard_sot%'] =  (df['standard_sot'].divide(non_penalty_sh) * 100.).round(1)
# to be considered leader in fbref have to have more than .395 shots per game / made it so 10+ shots too
mask_null_shot = (non_penalty_sh.divide(df.playing_time_mp) < .395) | (non_penalty_sh < 10)
df.loc[mask_null_shot, ['standard_g/sh', 'expected_npxg/sh', 'standard_sot%']] = np.nan
# to be considered leader in fbref have to have more than .111 sots on target per game / made it so 4+ shots too
mask_null_sot = (df.standard_sot.divide(df.playing_time_mp) < .111) | (df.standard_sot < 4)
df.loc[mask_null_sot, ['standard_g/sot']] = np.nan

Standardise columns to per 90 stats if not ratio/ percent stats

In [6]:
cols_to_standardise = list(df.columns[15:-15])
df[cols_to_standardise] = (df[cols_to_standardise].divide(df.playing_time_min, axis='rows') * 90.).round(3)

Save the combined dataset

In [7]:
df.to_parquet(os.path.join('data', 'fbref_combined.parquet'))